## More Lions is All You Need

running scoring and voting using `llama2` model via `ollama` service

In [1]:
from IPython.display import Markdown
from timeit import default_timer as timer

In [2]:
system = "you are a helpful assistant."
instruct = "think step by step, concisely yet comprehensively explain the balance sheet. return around 200 words"

The goal is to find the best answer from 5 generations of the same questions

In [3]:
from lionagi import Services, Session
from lionagi.libs import func_call

In [4]:
durations = []


async def explain(i):
    service = Services.Ollama(model="llama2")
    session = Session(system=system, service=service)

    # use timed call to get the runtime of the given function
    _, time_ = await func_call.tcall(
        func=session.chat, instruction=instruct, timing=True
    )

    durations.append(time_)
    return session

In [5]:
start_time = timer()

In [6]:
writer_sessions = await func_call.alcall([i for i in range(5)], explain)

In [7]:
run_time = timer() - start_time
print(f"\nClock time: {run_time:.2f} seconds")


Clock time: 48.51 seconds


In [8]:
import numpy as np

run_describe = f"""
Average runtime: {np.average(durations):.2f} seconds\n
Total runtime: {np.sum(durations):.2f} seconds\n
Total number of runs: {len(durations)}
"""
print(run_describe)


Average runtime: 36.24 seconds

Total runtime: 181.21 seconds

Total number of runs: 5



In [9]:
print(f"speed up { np.sum(durations) / float(run_time): .02f}")

speed up  3.74


### Get a group of critics

In [10]:
system = """
    as a highly critical professor, score an answer, only return a 
    positive integer
"""

instruction = """
    meticulously and critically give a score between 0-10 on the 
    following explaination of balance sheet.
"""

In [11]:
durations = []
results = []


async def score(context):
    service = Services.Ollama(model="llama2")

    # set up a function for one scorer to evaluate
    async def _inner(i):
        session = Session(system=system, service=service)

        out_, time_ = await func_call.tcall(
            func=session.chat, instruction=instruction, context=context, timing=True
        )

        durations.append(time_)
        return out_

    scores = await func_call.alcall(
        [i for i in range(4)], _inner
    )  # run 5 scorers in parallel

    # map the context with scores,
    out = {"context": context, "scores": scores}

    results.append(out)

    return (scores, context)

In [12]:
# get the responses from all previous sessions

responses = func_call.lcall(
    writer_sessions, lambda s: s.last_response_content["response"]
)

this is 15 runs in total across all responses,
so will be pretty slow

to speed up, reduce the number of responses


In [13]:
start_time = timer()

In [14]:
outputs = await func_call.alcall(responses, score)

In [15]:
run_time = timer() - start_time
print(f"\nClock time: {run_time:.2f} seconds")


Clock time: 160.83 seconds


In [16]:
scores = [o[0] for o in outputs]
contexts = [o[1] for o in outputs]

In [17]:
run_time = f"""
Average runtime: {np.average(durations):.2f} seconds\n
Total runtime: {np.sum(durations):.2f} seconds\n
Total number of runs: {len(durations)}
"""
print(run_time)


Average runtime: 108.72 seconds

Total runtime: 2174.49 seconds

Total number of runs: 20



In [18]:
import lionagi.libs.ln_convert as convert

avgs = []
for i in scores:
    z = []
    for j in i:
        try:
            a = convert.to_num(j)
            z.append(a)
        except:
            pass
    avgs.append(np.average(z))

print(avgs)

[8.0, 8.0, 6.25, 8.5, 8.25]


In [19]:
high_index = np.where(np.array(avgs) == max(avgs))
low_index = np.where(np.array(avgs) == min(avgs))

In [20]:
high_index

(array([3]),)

In [21]:
low_index

(array([2]),)

### High Score

In [24]:
print(f"High score: {max(avgs)}")
Markdown(contexts[3])

High score: 8.5



As a helpful assistant, I'd be delighted to explain the balance sheet in a clear and concise manner! Here's a step-by-step guide:

1. What is a Balance Sheet?
A balance sheet is a financial statement that provides a snapshot of a company's financial position at a specific point in time, usually at the end of an accounting period (e.g., quarterly or annually). It lists the company's assets, liabilities, and equity.
2. Assets: Definition & Classification
Assets are resources owned or controlled by the company that have economic value. Common asset categories include:
	* Current Assets: Cash, accounts receivable, inventory, prepaid expenses, etc., which are expected to be converted into cash within one year or less.
	* Non-Current Assets: Property, plant, and equipment (PPE), investments, and other assets that are not expected to be converted into cash within the next year.
3. Liabilities: Definition & Classification
Liabilities are debts or obligations owed by the company to external parties. Common liability categories include:
	* Current Liabilities: Accounts payable, taxes payable, and other debts that are expected to be paid within one year or less.
	* Non-Current Liabilities: Long-term loans, mortgages, and other liabilities that are not expected to be paid within the next year.
4. Equity: Definition & Classification
Equity represents the ownership interest in the company. Common equity categories include:
	* Common Stock: Shares held by shareholders, which represent ownership in the company.
	* Retained Earnings: Profits reinvested in the business, increasing the owner's equity.
5. Balance Sheet Accounts
The balance sheet typically includes several accounts, including:
	* Assets: Cash, receivables, inventory, prepaid expenses, property, plant, and equipment (PPE), investments, etc.
	* Liabilities: Accounts payable, taxes payable, long-term loans, mortgages, etc.
	* Equity: Common stock, retained earnings, etc.
6. Balance Sheet Analysis
Analyzing the balance sheet helps understand a company's financial position and performance. Key metrics include:
	* Liquidity Ratios: Examining the ability to pay short-term debts (e.g., current ratio) or generate cash from assets (e.g., cash conversion cycle).
	* Profitability Ratios: Assessing the company's ability to generate profits (e.g., return on assets, return on equity).
	* Solvency Ratios: Evaluating the company's ability to meet its long-term obligations (e.g., debt-to-equity ratio, interest coverage ratio).
7. Interpretation & Use Cases
Interpreting the balance sheet helps investors, creditors, and other stakeholders evaluate a company's financial health and make informed decisions. For example, a high level of assets compared to liabilities may indicate a strong financial position, while a low profit margin may suggest inefficiencies in operations.

In conclusion, the balance sheet is a crucial financial statement that provides insights into a company's financial position and performance. By understanding the different account categories, analyzing key metrics, and interpreting the results, stakeholders can make informed decisions and evaluate a company's financial health.

### Low score

In [23]:
print(f"Low score: {min(avgs)}")
Markdown(contexts[2])

Low score: 6.25



As a helpful assistant, I'd be delighted to explain the balance sheet in a clear and concise manner! Here it goes:

The balance sheet is a financial statement that presents the financial position of a company at a specific point in time. It provides a snapshot of the company's assets, liabilities, and equity. The balance sheet is organized into two sides: the asset side and the liability side.

On the asset side, you'll find the following categories:

1. Current Assets: These are assets that can be converted into cash within a year or less. Examples include cash, accounts receivable, inventory, and prepaid expenses.
2. Non-Current Assets: These are assets that cannot be converted into cash within a year. Examples include property, plant, and equipment (PPE), investments, and intangible assets like patents and trademarks.

On the liability side, you'll find the following categories:

1. Current Liabilities: These are liabilities that must be paid within a year or less. Examples include accounts payable, accrued expenses, and short-term loans.
2. Non-Current Liabilities: These are liabilities that cannot be paid within a year. Examples include long-term loans, mortgages, and pension obligations.
3. Equity: This represents the ownership interest in the company. It includes shareholders' equity, which is the residual interest after deducting total liabilities from total assets.

In summary, the balance sheet provides a snapshot of a company's financial position by listing its assets, liabilities, and equity. It helps investors, creditors, and other stakeholders evaluate the company's financial health and make informed decisions.

### Others

In [25]:
print(f"High score: {avgs[0]}")
Markdown(contexts[0])

High score: 8.0



As a helpful assistant, I'd be happy to explain the balance sheet in a clear and concise manner! Here it goes:

The balance sheet is a financial statement that provides a snapshot of a company's financial position at a particular point in time. It lists the company's assets, liabilities, and equity, and helps to understand how well-positioned the company is financially.

Assets: These are the things that the company owns or has control over, such as cash, inventory, property, plant, and equipment. The balance sheet lists the value of these assets and how they are classified into different categories.

Liabilities: These are the debts or obligations that the company owes to others, such as loans, accounts payable, and taxes payable. The balance sheet lists the amount of each liability and when it is due.

Equity: This represents the ownership interest in the company, including shareholder equity and retained earnings. Equity shows the amount of money that would be left over for shareholders if the company were to liquidate its assets and pay off its liabilities.

When looking at a balance sheet, it's important to consider the following:

1. Assets must equal Liabilities plus Equity (known as the accounting equation).
2. The balance sheet should be organized into different sections, such as current assets, non-current assets, current liabilities, and non-current liabilities.
3. The values of assets and liabilities should be accurately stated based on their fair market value.
4. The equity section should include the number of shares outstanding and the par value of each share.

By understanding these concepts, you can use the balance sheet to evaluate a company's financial health and make informed investment decisions.

In [26]:
print(f"High score: {avgs[1]}")
Markdown(contexts[2])

High score: 8.0



As a helpful assistant, I'd be delighted to explain the balance sheet in a clear and concise manner! Here it goes:

The balance sheet is a financial statement that presents the financial position of a company at a specific point in time. It provides a snapshot of the company's assets, liabilities, and equity. The balance sheet is organized into two sides: the asset side and the liability side.

On the asset side, you'll find the following categories:

1. Current Assets: These are assets that can be converted into cash within a year or less. Examples include cash, accounts receivable, inventory, and prepaid expenses.
2. Non-Current Assets: These are assets that cannot be converted into cash within a year. Examples include property, plant, and equipment (PPE), investments, and intangible assets like patents and trademarks.

On the liability side, you'll find the following categories:

1. Current Liabilities: These are liabilities that must be paid within a year or less. Examples include accounts payable, accrued expenses, and short-term loans.
2. Non-Current Liabilities: These are liabilities that cannot be paid within a year. Examples include long-term loans, mortgages, and pension obligations.
3. Equity: This represents the ownership interest in the company. It includes shareholders' equity, which is the residual interest after deducting total liabilities from total assets.

In summary, the balance sheet provides a snapshot of a company's financial position by listing its assets, liabilities, and equity. It helps investors, creditors, and other stakeholders evaluate the company's financial health and make informed decisions.

In [27]:
print(f"High score: {avgs[4]}")
Markdown(contexts[3])

High score: 8.25



As a helpful assistant, I'd be delighted to explain the balance sheet in a clear and concise manner! Here's a step-by-step guide:

1. What is a Balance Sheet?
A balance sheet is a financial statement that provides a snapshot of a company's financial position at a specific point in time, usually at the end of an accounting period (e.g., quarterly or annually). It lists the company's assets, liabilities, and equity.
2. Assets: Definition & Classification
Assets are resources owned or controlled by the company that have economic value. Common asset categories include:
	* Current Assets: Cash, accounts receivable, inventory, prepaid expenses, etc., which are expected to be converted into cash within one year or less.
	* Non-Current Assets: Property, plant, and equipment (PPE), investments, and other assets that are not expected to be converted into cash within the next year.
3. Liabilities: Definition & Classification
Liabilities are debts or obligations owed by the company to external parties. Common liability categories include:
	* Current Liabilities: Accounts payable, taxes payable, and other debts that are expected to be paid within one year or less.
	* Non-Current Liabilities: Long-term loans, mortgages, and other liabilities that are not expected to be paid within the next year.
4. Equity: Definition & Classification
Equity represents the ownership interest in the company. Common equity categories include:
	* Common Stock: Shares held by shareholders, which represent ownership in the company.
	* Retained Earnings: Profits reinvested in the business, increasing the owner's equity.
5. Balance Sheet Accounts
The balance sheet typically includes several accounts, including:
	* Assets: Cash, receivables, inventory, prepaid expenses, property, plant, and equipment (PPE), investments, etc.
	* Liabilities: Accounts payable, taxes payable, long-term loans, mortgages, etc.
	* Equity: Common stock, retained earnings, etc.
6. Balance Sheet Analysis
Analyzing the balance sheet helps understand a company's financial position and performance. Key metrics include:
	* Liquidity Ratios: Examining the ability to pay short-term debts (e.g., current ratio) or generate cash from assets (e.g., cash conversion cycle).
	* Profitability Ratios: Assessing the company's ability to generate profits (e.g., return on assets, return on equity).
	* Solvency Ratios: Evaluating the company's ability to meet its long-term obligations (e.g., debt-to-equity ratio, interest coverage ratio).
7. Interpretation & Use Cases
Interpreting the balance sheet helps investors, creditors, and other stakeholders evaluate a company's financial health and make informed decisions. For example, a high level of assets compared to liabilities may indicate a strong financial position, while a low profit margin may suggest inefficiencies in operations.

In conclusion, the balance sheet is a crucial financial statement that provides insights into a company's financial position and performance. By understanding the different account categories, analyzing key metrics, and interpreting the results, stakeholders can make informed decisions and evaluate a company's financial health.